# Feature Engineering
This notebook will create a small set of robust, cross-dataset features derived from the canonical features.

## Goals
1. Normalize raw flow counts by time to capture traffic intensity (e.g., packets per second, bytes per second)
2. Capture flow density characteristics (e.g., average bytes per packet)
3. Encode directional asymmetry in traffic behavior (e.g., packet-level direction ratio, byte-level direction ratio)

In [1]:
!pip -q install "PyAthena[SQLAlchemy]" sqlalchemy s3fs

In [2]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Display settings
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Connect to Athena

In [3]:
sess = sagemaker.Session()
region = boto3.Session().region_name

results_bucket = sess.default_bucket()
athena_results_path = f"s3://{results_bucket}/athena/staging/"

database_name = "aai540_eda"

engine = create_engine(
    f"awsathena+rest://@athena.{region}.amazonaws.com:443/{database_name}",
    connect_args={"s3_staging_dir": athena_results_path, "region_name": region},
)
print("Region:", region)
print("Athena results:", athena_results_path)

Region: us-east-1
Athena results: s3://sagemaker-us-east-1-776673915827/athena/staging/


In [4]:
# Helper functions for queries
def exec_ddl(sql: str):
    with engine.begin() as conn:
        conn.execute(text(sql))

def read_sql(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, engine)

## Verify merged dataset

In [5]:
merged_dataset = read_sql(f"""
SELECT *
FROM {database_name}.merged_canonical_normalized
Limit 5
""")
merged_dataset

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,label,original_attack_type,attack_category,source_dataset
0,0.366705,58,7952,28,30,4872,3080,0,,Normal,UNSW-NB15
1,2.302576,248,22654,122,126,7814,14840,0,,Normal,UNSW-NB15
2,0.005218,36,6808,18,18,4352,2456,0,,Normal,UNSW-NB15
3,0.014091,70,15518,34,36,2230,13288,0,,Normal,UNSW-NB15
4,0.001063,4,324,2,2,146,178,0,,Normal,UNSW-NB15


## Engineered Features
1. **`pkt_rate`** (*`pkt_rate = pkt_total / duration`*): Number of packets transmitted per second during the flow.
2. **`byte_rate`** (*`byte_rate = bytes_total / duration`*): Total bytes transferred per second during the flow.
3. **`bytes_per_pkt`** (*`bytes_per_pkt = bytes_total / (pkt_total + 1)`*): Average number of bytes carried per packet in the flow.
4. **`pkt_ratio`** (*`pkt_ratio = pkt_fwd / (pkt_bwd + 1)`*): Ratio of forward packets to backward packets in the flow.
5. **`byte_ratio`** (*`byte_ratio = bytes_fwd / (bytes_bwd + 1)`*): Ratio of forward bytes to backward bytes in the flow.

## **`pkt_rate`** (*`pkt_rate = pkt_total / duration`*)

This creates a new table version: `merged_canonical_normalized_v1`

In [6]:
# drop table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v1")

# delete S3 data directory to avoid HIVE_PATH_ALREADY_EXISTS error
import subprocess
subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v1/", "--recursive"], check=False)

# create table pkt_rate
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v1
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v1/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN duration IS NULL OR duration <= 0 THEN NULL
    ELSE CAST(pkt_total AS DOUBLE) / CAST(duration AS DOUBLE)
  END AS pkt_rate
FROM {database_name}.merged_canonical_normalized
""")

delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v1/20260213_180833_00052_ng26p_8645e280-2673-4f78-a6ca-6faab839b907
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v1/20260213_180833_00052_ng26p_4442a646-9f7d-4b81-8a42-7ec1f6447bc0


In [7]:
read_sql(f"""
SELECT duration, pkt_total, pkt_rate
FROM {database_name}.merged_canonical_normalized_v1
pkt_total
Limit 25
""")

,duration,pkt_total,pkt_rate
0,0.016101,10,621.079436
1,0.002814,12,4264.392324
2,0.003118,11,3527.902502
3,0.002219,10,4506.534475
4,0.002673,10,3741.114852
5,0.002246,13,5788.067676
6,0.001732,10,5773.672055
7,0.001719,10,5817.335660
8,0.001311,10,7627.765065
9,0.003075,13,4227.642276


### Sanity check

In [8]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN duration IS NULL OR duration <= 0 THEN 1 ELSE 0 END) AS bad_duration_rows,
  SUM(CASE WHEN (duration IS NULL OR duration <= 0) AND pkt_rate IS NOT NULL THEN 1 ELSE 0 END) AS pkt_rate_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v1
""")


,rows_total,bad_duration_rows,pkt_rate_should_be_null_but_isnt
0,26708942,5325854,0


## **`byte_rate`** (*`byte_rate = bytes_total / duration`*)

This creates a new table version: `merged_canonical_normalized_v2`

In [9]:
# drop v2 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v2")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v2/", "--recursive"], check=False)


# Create v2 with byte_rate (built on v1 so pkt_rate is retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v2
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v2/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN duration IS NULL OR duration <= 0 THEN NULL
    ELSE CAST(bytes_total AS DOUBLE) / CAST(duration AS DOUBLE)
  END AS byte_rate
FROM {database_name}.merged_canonical_normalized_v1
""")

delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v2/20260213_180856_00151_bi7xg_677c528d-b045-4b24-8a2b-8ad4d78ca4f7
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v2/20260213_180856_00151_bi7xg_8922f174-fc96-41d8-8dd1-e103585ac669


In [10]:
read_sql(f"""
SELECT duration, bytes_total, byte_rate, pkt_total, pkt_rate
FROM {database_name}.merged_canonical_normalized_v2
WHERE duration IS NOT NULL
LIMIT 25
""")

,duration,bytes_total,byte_rate,pkt_total,pkt_rate
0,0.000000,0,NaN,1,NaN
1,0.000006,0,0.000000,2,333333.333333
2,0.000000,0,NaN,1,NaN
3,0.000000,0,NaN,1,NaN
4,0.000000,0,NaN,1,NaN
5,0.000000,0,NaN,1,NaN
6,0.000000,0,NaN,1,NaN
7,0.000000,0,NaN,1,NaN
8,0.000000,0,NaN,1,NaN
9,0.000000,0,NaN,1,NaN


### Sanity check

In [11]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN duration IS NULL OR duration <= 0 THEN 1 ELSE 0 END) AS bad_duration_rows,
  SUM(CASE WHEN (duration IS NULL OR duration <= 0) AND byte_rate IS NOT NULL THEN 1 ELSE 0 END) AS byte_rate_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v2
""")


,rows_total,bad_duration_rows,byte_rate_should_be_null_but_isnt
0,26708942,5325854,0


## **`bytes_per_pkt`** (*`bytes_per_pkt = bytes_total / (pkt_total + 1)`*)

This creates a new table version: `merged_canonical_normalized_v3`

In [12]:
# drop v3 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v3")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v3/", "--recursive"], check=False)

# create v3 with bytes_per_pkt (built on v2 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v3
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v3/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_total IS NULL OR pkt_total <= 0 THEN NULL
    ELSE CAST(bytes_total AS DOUBLE) / CAST(pkt_total AS DOUBLE)
  END AS bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v2
""")

delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v3/20260213_180918_00124_9zhf7_b05f55e1-d858-4db8-9652-ebb85f9db38c
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v3/20260213_180918_00124_9zhf7_6a64ed4b-5e31-4787-bb50-9d333d36fd55
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v3/20260213_180918_00124_9zhf7_039ef4c8-5a38-4848-9b68-988576b27a7f


In [13]:
read_sql(f"""
SELECT duration, pkt_total, bytes_total, bytes_per_pkt
FROM {database_name}.merged_canonical_normalized_v3
WHERE bytes_total > 0
LIMIT 25
""")

,duration,pkt_total,bytes_total,bytes_per_pkt
0,61.016719,4,1,0.250000
1,60.062202,3,1,0.333333
2,60.324803,4,1,0.250000
3,0.000022,5,180,36.000000
4,60.222362,4,1,0.250000
5,116.083827,23,2053,89.260870
6,60.124963,4,1,0.250000
7,61.053266,4,1,0.250000
8,60.969532,4,1,0.250000
9,60.129240,4,1,0.250000


### Sanity check

In [14]:
read_sql(f"""
SELECT
  COUNT(*) AS rows_total,
  SUM(CASE WHEN pkt_total IS NULL OR pkt_total <= 0 THEN 1 ELSE 0 END) AS bad_pkt_total_rows,
  SUM(CASE WHEN (pkt_total IS NULL OR pkt_total <= 0) AND bytes_per_pkt IS NOT NULL THEN 1 ELSE 0 END) AS bytes_per_pkt_should_be_null_but_isnt
FROM {database_name}.merged_canonical_normalized_v3
""")

,rows_total,bad_pkt_total_rows,bytes_per_pkt_should_be_null_but_isnt
0,26708942,115260,0


## **`pkt_ratio`** (*`pkt_ratio = pkt_fwd / (pkt_bwd + 1)`*)

This creates a new table version: `merged_canonical_normalized_v4`

In [15]:
# drop v4 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v4")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v4/", "--recursive"], check=False)

# create v4 with pkt_ratio (built on v3 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v4
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v4/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN pkt_fwd IS NULL OR pkt_fwd < 0 THEN NULL
    WHEN pkt_bwd IS NULL OR pkt_bwd < 0 THEN NULL
    ELSE CAST(pkt_fwd AS DOUBLE) / (CAST(pkt_bwd AS DOUBLE) + 1.0)
  END AS pkt_ratio
FROM {database_name}.merged_canonical_normalized_v3
""")

delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v4/20260213_180938_00034_b5jna_872384f3-c2f6-4202-98c0-edca20a08528
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v4/20260213_180938_00034_b5jna_4d22062d-8f2c-440f-bba9-142f226d888f
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v4/20260213_180938_00034_b5jna_1e4e19cc-0a42-4197-90b6-0cccbab4ec32


In [16]:
read_sql(f"""
SELECT duration, pkt_total, pkt_fwd, pkt_bwd, pkt_ratio
FROM {database_name}.merged_canonical_normalized_v4
LIMIT 25
""")

,duration,pkt_total,pkt_fwd,pkt_bwd,pkt_ratio
0,0.000122,2,1,1,0.5
1,0.000141,2,1,1,0.5
2,0.000132,2,1,1,0.5
3,0.000116,2,1,1,0.5
4,0.000019,2,1,1,0.5
5,0.000001,2,1,1,0.5
6,0.000193,2,1,1,0.5
7,0.000128,2,1,1,0.5
8,0.002097,2,1,1,0.5
9,0.000265,2,1,1,0.5


## **`byte_ratio`** (*`byte_ratio = bytes_fwd / (bytes_bwd + 1)`*)

This creates a new table version: `merged_canonical_normalized_v5`

In [17]:
# drop v5 if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_canonical_normalized_v5")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_canonical_normalized_v5/", "--recursive"], check=False)

# Create v5 with byte_ratio (built on v4 so prior features are retained)
exec_ddl(f"""
CREATE TABLE {database_name}.merged_canonical_normalized_v5
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_canonical_normalized_v5/',
  parquet_compression = 'SNAPPY'
) AS
SELECT
  *,
  CASE
    WHEN bytes_fwd IS NULL OR bytes_fwd < 0 THEN NULL
    WHEN bytes_bwd IS NULL OR bytes_bwd < 0 THEN NULL
    ELSE CAST(bytes_fwd AS DOUBLE) / (CAST(bytes_bwd AS DOUBLE) + 1.0)
  END AS byte_ratio
FROM {database_name}.merged_canonical_normalized_v4
""")

delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v5/20260213_180955_00070_bn7mg_9388031a-fe0b-46b1-ae8a-3ac76d08a3a9
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v5/20260213_180955_00070_bn7mg_7ce2c495-077a-419f-b70a-a5573656678b
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/merged_canonical_normalized_v5/20260213_180955_00070_bn7mg_07be39a7-b9f2-4797-b6f6-a220ac5c07f5


In [18]:
read_sql(f"""
SELECT
  bytes_fwd,
  bytes_bwd,
  byte_ratio
FROM {database_name}.merged_canonical_normalized_v5
LIMIT 25
""")

,bytes_fwd,bytes_bwd,byte_ratio
0,394,0,394.000000
1,527,483,1.088843
2,382,483,0.789256
3,404,483,0.834711
4,2080,1129,1.840708
5,335,483,0.692149
6,2821,1377,2.047170
7,2149,1152,1.863833
8,431,483,0.890496
9,522,483,1.078512


## Finalize Feature Engineering Table

In [19]:
# drop final table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.feature_engineered")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/feature_engineered/", "--recursive"], check=False)

# create final feature_engineered table from v5
exec_ddl(f"""
CREATE TABLE {database_name}.feature_engineered
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/feature_engineered/',
  parquet_compression = 'SNAPPY'
) AS
SELECT *
FROM {database_name}.merged_canonical_normalized_v5
""")


delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/feature_engineered/20260213_181015_00061_iis8e_9292b010-f579-4698-ac1a-a767bf197f62
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/feature_engineered/20260213_181015_00061_iis8e_244dcbf0-ae63-4d83-b226-c14eaec6cf2a
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/feature_engineered/20260213_181015_00061_iis8e_0dcdbf9d-b014-4e95-a2f8-0363bb38c192


In [20]:
read_sql(f"""
SELECT
  *
FROM {database_name}.feature_engineered
LIMIT 10
""")

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,label,original_attack_type,attack_category,source_dataset,pkt_rate,byte_rate,bytes_per_pkt,pkt_ratio,byte_ratio
0,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
1,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
2,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
3,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
4,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
5,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
6,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
7,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
8,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0
9,0.0,1,0,1,0,0,0,1,scanning,Reconnaissance,TON_IoT,None,None,0.0,1.0,0.0


## Data integrity check

In [21]:
profile_invalid = read_sql(f"""
SELECT
  COUNT(*) AS total_rows,

  -- NULL counts
  SUM(CASE WHEN duration  IS NULL THEN 1 ELSE 0 END) AS duration_null,
  SUM(CASE WHEN pkt_total IS NULL THEN 1 ELSE 0 END) AS pkt_total_null,
  SUM(CASE WHEN bytes_total IS NULL THEN 1 ELSE 0 END) AS bytes_total_null,
  SUM(CASE WHEN pkt_fwd   IS NULL THEN 1 ELSE 0 END) AS pkt_fwd_null,
  SUM(CASE WHEN pkt_bwd   IS NULL THEN 1 ELSE 0 END) AS pkt_bwd_null,
  SUM(CASE WHEN bytes_fwd IS NULL THEN 1 ELSE 0 END) AS bytes_fwd_null,
  SUM(CASE WHEN bytes_bwd IS NULL THEN 1 ELSE 0 END) AS bytes_bwd_null,

  -- Negative counts
  SUM(CASE WHEN duration  < 0 THEN 1 ELSE 0 END) AS duration_neg,
  SUM(CASE WHEN pkt_total < 0 THEN 1 ELSE 0 END) AS pkt_total_neg,
  SUM(CASE WHEN bytes_total < 0 THEN 1 ELSE 0 END) AS bytes_total_neg,
  SUM(CASE WHEN pkt_fwd   < 0 THEN 1 ELSE 0 END) AS pkt_fwd_neg,
  SUM(CASE WHEN pkt_bwd   < 0 THEN 1 ELSE 0 END) AS pkt_bwd_neg,
  SUM(CASE WHEN bytes_fwd < 0 THEN 1 ELSE 0 END) AS bytes_fwd_neg,
  SUM(CASE WHEN bytes_bwd < 0 THEN 1 ELSE 0 END) AS bytes_bwd_neg,

  -- Zero counts
  SUM(CASE WHEN duration  = 0 THEN 1 ELSE 0 END) AS duration_zero,
  SUM(CASE WHEN pkt_total = 0 THEN 1 ELSE 0 END) AS pkt_total_zero,
  SUM(CASE WHEN bytes_total = 0 THEN 1 ELSE 0 END) AS bytes_total_zero

FROM {database_name}.feature_engineered
""")
profile_invalid

,total_rows,duration_null,pkt_total_null,bytes_total_null,pkt_fwd_null,pkt_bwd_null,bytes_fwd_null,bytes_bwd_null,duration_neg,pkt_total_neg,bytes_total_neg,pkt_fwd_neg,pkt_bwd_neg,bytes_fwd_neg,bytes_bwd_neg,duration_zero,pkt_total_zero,bytes_total_zero
0,26708942,0,0,0,0,0,0,0,115,0,0,0,0,0,0,5325739,115260,15981998


## Count rows to be dropped

In [22]:
invalid_row_summary = read_sql(f"""
SELECT
  COUNT(*) AS total_rows,
  SUM(
    CASE WHEN
      -- NULL raw features
      duration IS NULL OR pkt_total IS NULL OR bytes_total IS NULL OR
      pkt_fwd IS NULL OR pkt_bwd IS NULL OR bytes_fwd IS NULL OR bytes_bwd IS NULL OR

      -- negative raw features
      duration < 0 OR pkt_total < 0 OR bytes_total < 0 OR
      pkt_fwd < 0 OR pkt_bwd < 0 OR bytes_fwd < 0 OR bytes_bwd < 0 OR

      -- zeros 
      duration = 0 OR pkt_total = 0
    THEN 1 ELSE 0 END
  ) AS rows_to_drop
FROM {database_name}.feature_engineered
""")

invalid_row_summary

,total_rows,rows_to_drop
0,26708942,5325854


## Create cleaned table (drop ALL invalid rows: NULL, negative, or zero)

In [23]:
# create a cleaned table
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.feature_engineered_cleaned")

subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/feature_engineered_cleaned/", "--recursive"], check=False)

exec_ddl(f"""
CREATE TABLE {database_name}.feature_engineered_cleaned
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/feature_engineered_cleaned/',
  parquet_compression = 'SNAPPY'
) AS
SELECT *
FROM {database_name}.feature_engineered
WHERE
  -- Raw canonical must exist
  duration IS NOT NULL
  AND pkt_total IS NOT NULL
  AND bytes_total IS NOT NULL
  AND pkt_fwd IS NOT NULL
  AND pkt_bwd IS NOT NULL
  AND bytes_fwd IS NOT NULL
  AND bytes_bwd IS NOT NULL

  -- Raw canonical must be non-negative
  AND duration >= 0
  AND pkt_total >= 0
  AND bytes_total >= 0
  AND pkt_fwd >= 0
  AND pkt_bwd >= 0
  AND bytes_fwd >= 0
  AND bytes_bwd >= 0

  -- Drop zero denominators
  AND duration > 0
  AND pkt_total > 0
""")

delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/feature_engineered_cleaned/20260213_181218_00016_kqfvu_d175ec5f-515b-402c-95f6-ea0ac492b55e
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/feature_engineered_cleaned/20260213_181218_00016_kqfvu_4095160d-ea14-413f-a68a-b1233b38c423
delete: s3://sagemaker-us-east-1-776673915827/aai540/processed/feature_engineered_cleaned/20260213_181218_00016_kqfvu_f0c58afd-362a-4ab5-9652-28857bc7141b


## Inspect 

In [24]:
invalid_rows_sample = read_sql(f"""
SELECT
  duration, pkt_total, bytes_total, pkt_fwd, pkt_bwd, bytes_fwd, bytes_bwd,
  pkt_rate, byte_rate, bytes_per_pkt, pkt_ratio, byte_ratio,
  label, source_dataset
FROM {database_name}.feature_engineered_cleaned
WHERE
  duration IS NULL OR pkt_total IS NULL OR bytes_total IS NULL OR
  pkt_fwd IS NULL OR pkt_bwd IS NULL OR bytes_fwd IS NULL OR bytes_bwd IS NULL OR
  duration < 0 OR pkt_total < 0 OR bytes_total < 0 OR
  pkt_fwd < 0 OR pkt_bwd < 0 OR bytes_fwd < 0 OR bytes_bwd < 0 OR
  duration = 0 OR pkt_total = 0
LIMIT 20
""")

invalid_rows_sample

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,pkt_rate,byte_rate,bytes_per_pkt,pkt_ratio,byte_ratio,label,source_dataset


## Clean Athena catalog

In [25]:
tables_to_drop = [
    "merged_canonical_normalized_v1",
    "merged_canonical_normalized_v2",
    "merged_canonical_normalized_v3",
    "merged_canonical_normalized_v4",
    "merged_canonical_normalized_v5",
]

for t in tables_to_drop:
    exec_ddl(f"DROP TABLE IF EXISTS {database_name}.{t}")